In [1]:
class Obj():
  def _repr_html_(self):
    return "<b id='asdf'>bold</b><script>console.log('hi'); console.log(document.body.innerText); document.querySelector('#asdf').innerHTML = 'new';</script>"
    # return { "text/html": "<b><script>alert('hi');</script>bold</b>" }

Obj()

In [2]:
import IPython
# IPython.get_ipython().find_user_code("Test.ipynb")

In [3]:
import inspect
inspect.getfile(inspect.currentframe())

'/var/folders/sw/ssd2bhc911x2p56s_28rk0jc0000gn/T/ipykernel_59169/3653608800.py'

In [4]:
ipy = IPython.get_ipython()
ipy.filename

'<ipython console>'

In [5]:
# %matplotlib widget
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipympl
import ipympl.backend_nbagg
import matplotlib.backend_bases
import random
import ipywidgets

count = []

# class Canvas(ipympl.backend_nbagg.Canvas):
#     def draw(self):
#         # self.figure.axes[0].text(random.random(), random.random(), self._message)
#         # print(self._message)
#         # START HERE: draw some sane annotation, e.g. a selection of the line.
#         super().draw()
    

# @matplotlib.backend_bases._Backend.export
# class _Backend_snp(ipympl.backend_nbagg._Backend):
#     FigureCanvas = Canvas
#     FigureManager = ipympl.backend_nbagg.FigureManager


mpl.use("nbAgg") # Have to first change to a different backend in order to get the classes to reload on the next line.
mpl.use("module://ipympl.backend_nbagg")
# mpl.use("module://" + __name__)

xs = np.linspace(0, 2 * np.pi, 20)
ys = np.sin(xs)
fig, ax = plt.subplots()
# ax.text(0.5,0.5,"hi",fontdict={...})
fig.suptitle("suptitle")
# lines = ax.plot(x, y, picker=True, pickradius=5)
# lines = ax.plot(xs, ys)
# ax.xaxis.mouseover = "hi"
# def onclick(event):
#     tx = 'button=%d, x=%d, y=%d, xdata=%f, ydata=%f' % (event.button, event.x, event.y, event.xdata, event.ydata)
#     ax.set_title(tx)

# def onpick(event):
#     thisline = event.artist
#     xdata = thisline.get_xdata()
#     ydata = thisline.get_ydata()
#     ind = event.ind
#     points = tuple(zip(xdata[ind], ydata[ind])) 
#     ax.set_title(points)
# cid = fig.canvas.mpl_connect('button_press_event', onclick)
# cid = fig.canvas.mpl_connect('pick_event', onpick)

import matplotlib.patheffects as path_effects
# hover_effect = path_effects.withSimplePatchShadow()
hover_effect = path_effects.Stroke(linewidth=3, foreground='black')

def add_hover_effect(artist):
    # if "color" in mpl.artist.ArtistInspector(artist).get_setters():
    # try:
    #     artist._orig_color = artist.get_color()
    #     artist.set_color((0.1, 0.2, 0.5))
    # except:
    #     pass
    artist.set_path_effects(artist.get_path_effects() + [hover_effect])

def remove_hover_effect(artist):
    # try:
    #     artist.set_color(artist._orig_color)
    # except:
    #     pass
    artist.set_path_effects([effect for effect in artist.get_path_effects() if effect is not hover_effect])

def selectable(artist):
    return not isinstance(artist, matplotlib.patches.Rectangle)

def children(artist):
    if "get_children" in dir(artist):
        if isinstance(artist, matplotlib.figure.Figure):
            return artist.get_children()[::-1] # reverse the children so we pick the deep stuff on mouseover
        else:
            return artist.get_children()
    else:
        return []

def snp_is_hovered(artist, mouseevent):
    if not selectable(artist):
        return False
    if isinstance(artist, matplotlib.axis.Tick):
        for child in children(artist):
            if snp_is_hovered(child, mouseevent):
                return True
        return False
    else:
        return artist.contains(mouseevent)[0]

def snp_hovered_artist(artist, mouseevent):
    for child in children(artist):
        hovered_descendent = snp_hovered_artist(child, mouseevent)
        if hovered_descendent is not None:
            return hovered_descendent
    if artist.contains(mouseevent)[0]:
        return artist
    else:
        return None

from time import time

last_hovered = None

def onmousemove(event):
    global last_hovered
    # if event.inaxes is None:
    #     return
    t1 = time()
    # fig_artists = all_artists(event.canvas.figure)
    hovered_artist = snp_hovered_artist(event.canvas.figure, event)
    # hovered_artists = [artist for artist in fig_artists if snp_is_hovered(artist, event)]
    # hovered_artists = [artist for artist in fig_artists if artist.contains(event)[0]]
    if last_hovered is not None:
        remove_hover_effect(last_hovered)
    # for artist in hovered_artists:
        # add_hover_effect(artist)
    if hovered_artist is not None:
        add_hover_effect(hovered_artist)
    
    duration = time() - t1
    # event.canvas.manager.toolmanager.send_message("asdf")
    title = str(hovered_artist._snp_names) if "_snp_names" in dir(hovered_artist) else str(hovered_artist)
    # event.canvas.manager.set_window_title(str(duration))
    event.canvas.manager.set_window_title(title)
    # try:
    #     event.canvas.manager.set_window_title(str(hovered_artist.get_bbox()))
    # except:
    #     pass
    # event.canvas.manager.set_window_title(str((event.x,event.y)))
    if last_hovered is not hovered_artist:
        event.canvas.draw_idle()
        last_hovered = hovered_artist
    # print(len(artists))
    # print(artists)    
        
        # print((event.xdata, event.ydata))
        # event.inaxes.add_artist(mpl.patches.Circle((event.xdata, event.ydata), 0.01))

fig.canvas.mpl_connect('motion_notify_event', onmousemove)
# fig.canvas.mpl_connect('figure_enter_event', lambda event: event.canvas.manager.set_window_title("enter"))
# fig.canvas.mpl_connect('figure_leave_event', lambda event: event.canvas.manager.set_window_title("leave"))


def flatten(lists):
    return sum(lists, []) # https://stackoverflow.com/a/952946

def all_artists(artist):
    if "get_children" in dir(artist):
        return [artist] + flatten([all_artists(artist) for artist in artist.get_children()])
    else:
        return [artist]


def print_artists(artist, prefix=""):
    print(prefix + str(artist))
    if "get_children" in dir(artist):
        for child in artist.get_children():
            print_artists(child, prefix + "    ")

# setattr(mpl.figure.Figure, "asdf", lambda self: print("asdf"))

# fig.get_children()[0].get_children()
# all_artists(fig)
# for artist in all_artists(fig):
#     if "get_bbox" in dir(artist):
#         bbox = artist.get_bbox()
#     elif "get_bbox_patch" in dir(artist):
#         bbox = artist.get_bbox_patch()
#     else:
#         bbox = None
#     print(artist, bbox)
# ax.set_ylabel("asdf")
# print(fig.canvas._repr_mimebundle_())
# plt.show(fig)
# print_artists(fig)
# ax.yaxis.label.get_window_extent()
# old_meth = fig.canvas.handle_event
# def handle_event(self, event):
#     print(event['type'])
#     return mpl.backends.backend_webagg_core.FigureCanvasWebAggCore.handle_event(self, event)
# setattr(ipympl.backend_nbagg.Canvas, "handle_event", handle_event)

# print_artists(fig) # need this after, so ticks are positions
# for artist in all_artists(fig):
#     if "get_bbox" in dir(artist):
#         bbox = artist.get_bbox()
#     elif "get_bbox_patch" in dir(artist):
#         bbox = artist.get_bbox_patch()
#     else:
#         bbox = None
#     print(artist, bbox)

# fig.canvas


In [6]:
%%javascript

define('snpInspectorWidget', ['@jupyter-widgets/base'], function(widgets) {
    var SNPWidgetView = widgets.DOMWidgetView.extend({
        render: function() {
            SNPWidgetView.__super__.render.apply(this, arguments);
            this._html_changed();
            this._js_changed();
            this.listenTo(this.model, 'change:html', this._html_changed, this);
            this.listenTo(this.model, 'change:js', this._js_changed, this);
        },

        _html_changed: function() {
            this.el.innerHTML = this.model.get('html');
        },

        _js_changed: function() {
            eval(this.model.get('js'));
        }
    });

    return {
        SNPWidgetView: SNPWidgetView
    }
});

<IPython.core.display.Javascript object>

In [7]:
import ipywidgets
import traitlets

class SNPWidget(ipywidgets.DOMWidget):
    _view_module = traitlets.Unicode('snpInspectorWidget').tag(sync=True)
    _view_module_version = traitlets.Unicode('0.1.0').tag(sync=True)
    _view_name = traitlets.Unicode('SNPWidgetView').tag(sync=True)
    html = traitlets.Unicode("<strong>asdfasdf</strong>").tag(sync=True)
    js   = traitlets.Unicode("").tag(sync=True)



In [36]:
# def default_snp_inspector_html(self):
#   out = '<div style="padding-left: 1em; border: solid 1px black">' + str(self)
#   if "get_children" in dir(self):
#     for child in self.get_children():
#       out += child.snp_inspector_html()
#   out += "</div>"
#   return out
  
def textbox(label, value):
  return "<label>" + label + '<input type="text" value="' + str(value) + '"></label>'

# def recurse(value):
  # if "figure_snp_inspector_html" in dir(value):

class GenericObj():
  pass

trivial_names = set(dir(GenericObj()))

def tag_with_path_deep(self, path_str):
  if hasattr(self, "_snp_names"):
    if path_str not in self._snp_names:
      self._snp_names.add(path_str)
    # print(entry)
  else:
    try:
      self._snp_names = {path_str}
      # print(entry)
    except:
      pass # can't set new attrs on primitives
  
  # out += textbox("_suptitle", self._suptitle)
  children = self.get_children()

  for name in dir(self):
    # if not name.startswith("_"):
    with mpl._api.deprecation.suppress_matplotlib_deprecation_warning():
      value = getattr(self, name)
    if not callable(value) and name not in trivial_names:
      if isinstance(value, list):
        for i, item in enumerate(value):
          if item in children:
            tag_with_path_deep(item, path_str + "." + name + "[" + str(i) + "]")
      else:
        if value in children:
          tag_with_path_deep(value, path_str + "." + name)

def snp_inspector_html(self):
  out = '<details style="padding-left: 1em; border: solid 1px black">'
  out += "<summary>" + str(self._snp_names) + "</summary>"
  out += str(self)
  # out += textbox("_suptitle", self._suptitle)
  children = self.get_children()

  visited_children = []

  for name in dir(self):
    # if not name.startswith("_"):
    with mpl._api.deprecation.suppress_matplotlib_deprecation_warning():
      value = getattr(self, name)
    if not callable(value) and name not in trivial_names:
      if isinstance(value, list):
        for i, item in enumerate(value):
          if item in children:
            if item not in visited_children:
              out += item.snp_inspector_html()
              visited_children.append(item)
          else:
            out += "<div>" + name  + "[" + str(i) + "]: " + str(item)[0:100] + "</div>"
      else:
        if value in children:
          if value not in visited_children:
            out += value.snp_inspector_html()
            visited_children.append(value)
        else:
          out += "<div>" + name + ": " + str(value)[0:100] + "</div>"

  out += "</details>"
  return out

setattr(matplotlib.artist.Artist, "snp_inspector_html", snp_inspector_html)
# setattr(matplotlib.figure.FigureBase, "snp_inspector_html", figure_snp_inspector_html)


# plt.show(fig)
snp = SNPWidget()
tag_with_path_deep(fig, "fig")
tag_with_path_deep(ax, "ax")
snp.html = fig.snp_inspector_html()
snp.js = """
// console.log(this.el)
console.log(document.body.querySelectorAll("canvas"))
"""
box = ipywidgets.HBox(children=[fig.canvas, snp], layout=ipywidgets.Layout(display="inline-flex", align_items = 'flex-start'))
box

In [9]:
dir(fig)
for name in dir(fig):
  if not name.startswith("_"):
    print()
    print()
    print("### " + name + " ###")
    print(getattr(fig, name).__doc__)



### add_artist ###

        Add an `.Artist` to the figure.

        Usually artists are added to Axes objects using `.Axes.add_artist`;
        this method can be used in the rare cases where one needs to add
        artists directly to the figure instead.

        Parameters
        ----------
        artist : `~matplotlib.artist.Artist`
            The artist to add to the figure. If the added artist has no
            transform previously set, its transform will be set to
            ``figure.transSubfigure``.
        clip : bool, default: False
            Whether the added artist should be clipped by the figure patch.

        Returns
        -------
        `~matplotlib.artist.Artist`
            The added artist.
        


### add_axes ###
Add an Axes to the figure.

Call signatures::

    add_axes(rect, projection=None, polar=False, **kwargs)
    add_axes(ax)

Parameters
----------
rect : sequence of float
    The dimensions [left, bottom, width, height] of the new Axes. Al

In [10]:
this_cell_code = IPython.get_ipython().history_manager.input_hist_raw[-1]
print(this_cell_code)

this_cell_code = IPython.get_ipython().history_manager.input_hist_raw[-1]
print(this_cell_code)


In [11]:
fig._suptitle._text

'suptitle'

In [12]:
import ast
import astor

# list(map(lambda node: ast.dump(node), ast.walk(ast.parse("fig.axes[0].xaxis.label"))))

print(astor.dump_tree(ast.parse("fig.axes[0].xaxis.label")))

astor.to_source(ast.parse("fig.axes[0].xaxis.label"))

Module(
    body=[
        Expr(
            value=Attribute(
                value=Attribute(
                    value=Subscript(value=Attribute(value=Name(id='fig'), attr='axes'),
                        slice=Constant(value=0, kind=None)),
                    attr='xaxis'),
                attr='label'))],
    type_ignores=[])


'fig.axes[0].xaxis.label\n'

In [13]:
def imported_names_shortest_first(module):
  glbls = globals()
  names = list(glbls.keys()) # need list to avoid errors about dict changing size during iteration
  return sorted([name for name in names if glbls[name] is module], key=len)

print(imported_names_shortest_first(matplotlib))
print(imported_names_shortest_first(matplotlib.pyplot))

['mpl', 'matplotlib']
['plt']


In [14]:
import ast
import astor

# https://ipython.readthedocs.io/en/stable/config/inputtransforms.html

def tag_name(parent, name, val):
  entry = (parent, name)
  if hasattr(val, "_snp_names"):
    if entry not in val._snp_names:
      val._snp_names.append(entry)
    # print(entry)
  else:
    try:
      val._snp_names = [entry]
      # print(entry)
    except:
      pass # can't set new attrs on primitives
  return val

def tag_root_name(name, val):
  return tag_name(None, name, val)

def propogate_tags(val):
  return _propogate_tags(set(), val)
  
def _propogate_tags(visited, val):
  if id(val) not in visited:
    visited.add(id(val))
    if hasattr(val, "__dict__"):
      for attr_name, child in vars(val).items():
          tag_name(val, attr_name, child)
          _propogate_tags(visited, child)


def tag(lines):
  code_in = "".join(lines)
  if "IPython" in code_in:
    print(code_in)
  return lines

IPython.get_ipython().input_transformers_post = []

# tag_name(None, "fig", fig)
# propogate_tags(fig)
# fig._suptitle._snp_names

In [15]:
this_cell_code = IPython.get_ipython().history_manager.input_hist_raw[-1]
print(this_cell_code)

this_cell_code = IPython.get_ipython().history_manager.input_hist_raw[-1]
print(this_cell_code)
